<a href="https://colab.research.google.com/github/rawatnikhil857/knightRiders-hackon/blob/main/context_question_answer_database_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing python libraries for data analysis

In [ ]:
import numpy as np
import pandas as pd

Reading the "movies.csv" and "ratings.csv" from MovieLens 20M Dataset which is accessible from https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset?select=movie.csv and
https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset?select=rating.csv
respectively

In [ ]:
movies = pd.read_csv("movie.csv")
ratings = pd.read_csv("rating.csv")
ratings

Code to map movieId with index and UserId with starting index for that user for easy processing

In [ ]:

map = {}
for index in movies.index:
  map[movies["movieId"][index]] = index;

map2 = {}
prev = 0;
for index in ratings.index:
  if(prev == 3000):
    break
  if(ratings["userId"][index] != prev):
    prev = ratings["userId"][index]
    map2[prev] = index

Function to filter top movies for a user and creating { context, question, answer } pairs for them

In [ ]:
def fun(userId):
  arr = []
  for index in range(map2[userId], ratings.index.stop):
    if ratings["userId"][index] == userId+1:
      break;
    movieId = ratings["movieId"][index]
    rating = ratings["rating"][index]
    pair = (rating, movieId)
    arr.append(pair)
  arr.sort(reverse=True)
  arr = arr[0:40]
  np.random.shuffle(arr)
  arr1 = arr[0:10]
  arr2 = arr[10:40]
  answer = ""
  context = ""
  for a,b in arr1:
    answer += movies["title"][map[b]] + ", " + movies["genres"][map[b]] + ", Rating: " + str(a) + "\n"
  for a,b in arr2:
    context += movies["title"][map[b]] + ", " + movies["genres"][map[b]] + ", Rating " + str(a) + "\n"
  question1 = "Given a user's past movie ratings in the format: Title, Genres, Rating.\nRatings range from 1.0 to 5.0\n\n"
  question2 = "What will be some more movies this user might like?"
  question1 += context
  return (question1, question2, answer)

Processing the dataset into our desired format for further training

In [ ]:
prompts = []
count = 0
index = 1
while(count > -1):
  if(count == 1000):
    break;
  if(map2[index+1] - map2[index] > 100):
    prompts.append(fun(index))
    count = count + 1
  index = index + 1

In [ ]:
df = pd.DataFrame(prompts, columns=['Context', 'Question', 'Answer'])
df

,Context,Question,Answer
0,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,Harry Potter and the Chamber of Secrets (2002)...
1,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Shining, The (1980), Horror, Rating: 5.0\nBett..."
2,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Gladiator (2000), Action|Adventure|Drama, Rati..."
3,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"American Haunting, An (2005), Drama|Horror|Mys..."
4,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Lagaan: Once Upon a Time in India (2001), Come..."
...,...,...,...
995,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Dark Knight Rises, The (2012), Action|Adventur..."
996,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Annie Hall (1977), Comedy|Romance, Rating: 5.0..."
997,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Beetlejuice (1988), Comedy|Fantasy, Rating: 5...."
998,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Arsenic and Old Lace (1944), Comedy|Mystery|Th..."


Exporting the processed data into CSV file for future use

In [ ]:
df.to_csv('Context-Question-AnswerDatabase.csv')

In [ ]:
df

,Context,Question,Answer
0,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Brotherhood of the Wolf (Pacte des loups, Le) ..."
1,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Blair Witch Project, The (1999), Drama|Horror|..."
2,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Titanic (1997), Drama|Romance, Rating: 5.0\nMi..."
3,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"War of the Worlds (2005), Action|Adventure|Sci..."
4,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Rosencrantz and Guildenstern Are Dead (1990), ..."
...,...,...,...
995,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Prestige, The (2006), Drama|Mystery|Sci-Fi|Thr..."
996,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"M*A*S*H (a.k.a. MASH) (1970), Comedy|Drama|War..."
997,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Lethal Weapon 4 (1998), Action|Comedy|Crime|Th..."
998,Given a user's past movie ratings in the forma...,What will be some more movies this user might ...,"Ben-Hur (1959), Action|Adventure|Drama, Rating..."
